In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn.datasets import fetch_openml
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler

data= fetch_openml('mnist_784', parser="auto", version=1)#Get data from https://www.openml.org/d/554
dfData = pd.DataFrame(np.c_[data["data"],data["target"]],columns = data["feature_names"]+["target"])

In [2]:
#creating a pipeline containing a MinMaxScaler
img_pipeline = Pipeline([("mm_scaler", MinMaxScaler())])

#define the target values
y = dfData["target"]

#drop the target values from the dataset X
dfData = dfData.drop("target",axis=1)
X = dfData.copy()
X_transf = img_pipeline.fit_transform(X)

In [3]:
stratSplit = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=0)

#splitting on the transformed dataset (defined before as X_transf)
#no need for iloc access because we are using a numpy array and not a pandas dataframe
for train_index, test_index in stratSplit.split(X_transf, y):
    X_train = X_transf[train_index]
    X_test = X_transf[test_index]
    
    y_train = y[train_index]
    y_test = y[test_index]

In [ ]:
#storing results in a cv function
log_reg = LogisticRegression(C=1e5, max_iter=100)
def performCV(log_reg, X_train, y_train):
    return cross_validate(log_reg,
                             X = X_train,
                             y = y_train,
                            scoring = "accuracy",
                            cv = 2,
                            n_jobs=-1,
                            verbose = False,
                            return_train_score=True,
                            return_estimator=True)
results = performCV(log_reg, X_train, y_train)

In [ ]:
#np.argmax(): is a NumPy function that returns the index of the maximum value in an array.
bestEstInd = np.argmax(results["test_score"])
best_estimator = results["estimator"][bestEstInd] #our defined index from the line above

#accesses the feature importance scores for the first class (assuming a classification problem).
#saving the coefficients
#coefficients are the weights of each feature we have
feature_importaces_zero = best_estimator.coef_[0]

In [ ]:
#10 rows = 10 different classes
#784 columns = number of features we have
print(best_estimator.coef_.shape)
best_estimator.coef_

In [ ]:
#creating a dataframe to display our features and weights/coefficients 
#+ absolute value of weights (to avoid negative values)
dfImp = pd.DataFrame({"feature":dfData.columns,
                     "weight":feature_importaces_zero,
                     "weightAbs":np.absolute(feature_importaces_zero)})

featureWeights = np.abs(dfImp["weight"].values)
# featureWeights[featureWeights<0.1]=0

#plotting the feature weights as an image
plt.imshow(featureWeights.reshape(28,28)) #28,28 as a size of the matrix: it is also the shape our images have
plt.axis('off')
plt.colorbar()
plt.title("Feature weight visualization")
plt.show()

In [ ]:
#cumulative sum of our weights
#creating a plot of how much % of our total sum of all the weight values that is contained in a certain number of features
#sorting weights by largest to smallest using the absolute values and doing a cumulative sum
#cumulative sum: what percentage of a total weight magnitude is contained up to a certain number of features
dfImpCumImp = dfImp.sort_values("weightAbs",ascending=False).reset_index(drop=True)
dfImpCumImp["cumSumWeights"] = dfImpCumImp["weightAbs"].cumsum()
dfImpCumImp["cumSumPerc"] = 100*dfImpCumImp["weightAbs"].cumsum()/dfImpCumImp["cumSumWeights"].iloc[-1]
plt.plot(dfImpCumImp.index,dfImpCumImp["cumSumPerc"])
plt.title("Cumulative weight contained in varying number of features")
plt.xlabel("Number of features")
plt.ylabel("Cumulative weight accounted for")
plt.show()
xPercTotalWeightsInd = dfImpCumImp.loc[dfImpCumImp["cumSumPerc"]<99].index[-1]

#looking at all features that contain 99% of our weight
print("# features that contain 99% of weight:",xPercTotalWeightsInd+1)
#The weight of the next feature (the one that, when added, would push the cumulative weight above 99%).
print("Weight of next feature:",dfImpCumImp.iloc[xPercTotalWeightsInd+1]["weight"])

In [ ]:
#looking at the weight
#value close to 0 = small, value away from 0 = bigger
dfImp.sort_values("weight", ascending=False, inplace=True)
print(dfImp.head())
print(dfImp.tail())

In [ ]:
#sorting: largest weights first
sorted(zip(feature_importaces_zero, dfData.columns), reverse=True)

In [ ]:
#now looking at all feature names where the weight is greater and smaller than -0.04
impFeat = dfImp.loc[(dfImp["weight"]>0.04)|(dfImp["weight"]<-0.04)]["feature"]
impFeat

In [ ]:
#now doing a stratified shuffle split on our 511 features only
X_transf_red = img_pipeline.fit_transform(X[impFeat])
for train_index, test_index in stratSplit.split(X_transf_red, y):
    X_train_red = X_transf_red[train_index]
    X_test_red = X_transf_red[test_index]
    
    y_train_red = y[train_index]
    y_test_red = y[test_index]

In [ ]:
results_red = performCV(log_reg, X_train_red, y_train_red)

In [ ]:
X_train_red.shape

In [ ]:
#comparing full set to the reduced one
def printCVPerformance(results,results_red):
    print("Number of features all:",results["estimator"][0].coef_.shape[1])
    print("Number of features reduced:",results_red["estimator"][0].coef_.shape[1])
    print("Test scores with all features:",round(np.mean(results["test_score"]),4))
    print("Test scores with reduced feature set:",round(np.mean(results_red["test_score"]),4))
    print()
    print("Average train time all features:",round(np.mean(results["fit_time"]),2))
    print("Average train time all features:",round(np.mean(results_red["fit_time"]),2))
printCVPerformance(results,results_red)

In [ ]:
#initialize model with iterations from 100 to 1000
#perform cv on training set and reduced training set
#and print out the performance features
for max_iter in [100,500,1000]:
    log_reg = LogisticRegression(C=1e5, max_iter=max_iter)
    results = performCV(log_reg, X_train, y_train)
    results_red = performCV(log_reg, X_train_red, y_train_red)
    print("Performance for max_iter:",max_iter)
    printCVPerformance(results,results_red)
    print("\n")
#result on the reduced training set: training time approx. 33% faster

#### Play around with the values e.g. 95% instead of 99% and see if it will improve the results!!! Always try to cut down features (e.g. less important features) to improve your speed and see if it keeps the accuracy on a significant level. Another way (later on) will be dimensionality reduction. If test scores drop it could be overfitting. Validation performance is dropping: also a starting sign for overfitting on the data set. 